In [1]:

#Installing all necessary packages 
!pip install snowflake-connector-python[pandas]
!pip install holidays
!pip install --upgrade snowflake-sqlalchemy
!pip install imblearn
!pip install catboost
!pip install probatus
from probatus.feature_elimination import ShapRFECV
!pip install tensorflow
!pip install ipywidgets
!pip install pandas_profiling
!pip install optuna
!pip3 install causallift


import optuna
from sklearn.metrics import mean_squared_error
from pandas_profiling import ProfileReport
import snowflake.connector
import numpy as np
import pandas as pd
import datetime as dt
import boto3
import base64
from numpy import dtype, isnan, sqrt
from sqlalchemy import create_engine
from sqlalchemy import pool
from sqlalchemy.dialects import registry
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import serialization
from botocore.exceptions import ClientError
from snowflake.connector.pandas_tools import write_pandas, pd_writer
from dateutil.easter import *
from pathlib import Path
from imblearn.over_sampling import SMOTE
import gc
import dask.dataframe as dd
import dask
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier
import catboost
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, log_loss
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.svm import SVC


import seaborn as sns
import shap
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from optuna.samplers import TPESampler
import catboost
import pickle

!jupyter nbextension enable --py widgetsnbextension

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached snowflake_connector_python-3.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.6 MB)
  Using cached asn1crypto-1.5.1-py2.py3-none-any.whl (105 kB)
  Using cached pycryptodomex-3.17-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached PyJWT-2.6.0-py3-none-any.whl (20 kB)
  Using cached oscrypto-1.3.0-py2.py3-none-any.whl (194 kB)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached holidays-0.21-py3-none-any.whl (230 kB)
  Using cached PyMeeus-0.5.12.tar.gz (5.8 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 9.6 MB/s eta 0:00:00
  Using cached hijri_converter-2.2.4-py3-none-any.whl (14 kB)
  Created wheel for PyMeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=4444de46ac6b683af16c3bff1a8193111edcbe036688e4d6a4e39c63fe648f6b
  Sto

`import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.


ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/sklearn/metrics/__init__.py)

In [2]:
# Connect to snowflake
ctx = snowflake.connector.connect(
      user='TWINKLE',
      account='mja29153.us-east-1',
      password = '@Luiwatson7',
      warehouse='PROGRESSIVE_LARGE',
      database='CLIENT',
      schema='PROGRESSIVE'
  )



In [3]:
cur = ctx.cursor()
sql = \
"""
 SELECT  s.*, d.CONVERSION_TYPE
 FROM CLIENT.PROGRESSIVE.REDS_IMPRESSIONS s
 JOIN CLIENT.PROGRESSIVE.REDS_CONVERSIONS d
 ON s.tdid = d.tdid
 WHERE s.TDID <> '00000000-0000-0000-0000-000000000000'
 AND s.DATE between '2023-02-01' and '2023-02-28'
 ORDER BY s.DATE DESC
 limit 1000000;

                  
"""
cur.execute(sql)

## Add limit 10 mil after date (before 'as a')
### Documentation 
# Fetch the result set from the cursor and deliver it as the Pandas DataFrame.


df = cur.fetch_pandas_all()

#for df in cur.fetch_pandas_all():df.to_pickle('df')



# # Fetch the result set from the cursor and deliver it as the Pandas DataFrame.
# #for df in cur.fetch_pandas_batches():
#  #   my_dataframe_processing_function(df)
cur.close()
ctx.close()

In [4]:
df.to_pickle('df_claritas')

In [2]:
df = pd.read_pickle('df_claritas')
#df = df.sample(frac=0.3)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 93 columns):
 #   Column                                      Non-Null Count    Dtype         
---  ------                                      --------------    -----         
 0   LOG_ENTRY_TIME                              1000000 non-null  datetime64[ns]
 1   IMPRESSION_ID                               1000000 non-null  object        
 2   PARTNER_ID                                  1000000 non-null  object        
 3   ADVERTISER_ID                               1000000 non-null  object        
 4   CAMPAIGN_ID                                 1000000 non-null  object        
 5   AD_GROUP_ID                                 1000000 non-null  object        
 6   PRIVATE_CONTRACT_ID                         467220 non-null   object        
 7   AUDIENCE_ID                                 988818 non-null   object        
 8   CREATIVE_ID                                 1000000 non-null  o

In [3]:
df.head()


,LOG_ENTRY_TIME,IMPRESSION_ID,PARTNER_ID,ADVERTISER_ID,CAMPAIGN_ID,AD_GROUP_ID,PRIVATE_CONTRACT_ID,AUDIENCE_ID,CREATIVE_ID,AD_FORMAT,...,PARTNER_NAME,PRIVATE_CONTRACT_NAME,SUPPLY_VENDOR_NAME,TTD_NATIVE_CONTEXT_TYPE_NAME,WEEK_OF_YEAR,TIME_OF_WEEK_NAME,METRO_NAME,GEO_SEGMENT_ID,GEO_SEGMENT_NAME,CONVERSION_TYPE
0,2023-02-28 13:42:31,fa443136-c8fa-4bf3-8b13-1dff8f0814f6,9uxl7z1,x0t0ev0,k15fygx,hqsaoth,5cx8jjn,v6xwvq8,tp6ooyp5,728x90,...,Progressive Insurance,None,None,None,9,Tuesday: 7:00,San Angelo TX,f9dbm6ppcs,United States - DMA - San Angelo TX,other:x8cxk4v
1,2023-02-28 17:57:41,b33ae292-9116-4bae-b8f1-50f9b84c8eb4,9uxl7z1,x0t0ev0,qteib1h,d1b10is,None,9tt9w1c,itaqr7t1,160x600,...,Progressive Insurance,None,None,None,9,Tuesday: 12:00,Miami-Ft. Lauderdale FL,j5vj1px9mq,United States - DMA - Miami-Ft. Lauderdale FL,other:x8cxk4v
2,2023-02-28 02:24:55,501370d3-b60d-4a99-a0e9-305e5895b473,9uxl7z1,x0t0ev0,k15fygx,78vezg8,None,3yfrfwa,u1p1v7c8,300x250,...,Progressive Insurance,None,None,None,9,Monday: 19:00,Salt Lake City UT,ir1ynhqwfk,United States - DMA - Salt Lake City UT,other:x8cxk4v
3,2023-02-28 01:19:36,e8d1b63a-6a0c-4205-a8ff-2f6adae987e4,9uxl7z1,x0t0ev0,qteib1h,h8k1u9p,woafcom,9tt9w1c,qaz8bi17,9544x9544,...,Progressive Insurance,YieldMo - Global - RON - Native Text & Image -...,None,None,9,Monday: 20:00,Orlando-Daytona Beach-Melbourne FL,pkxadl3lb9,United States - DMA - Orlando-Daytona Beach-Me...,other:x8cxk4v
4,2023-02-28 02:20:08,4d9d0aa2-8ac8-4968-9d29-83c9c10dd5ef,9uxl7z1,r4ww1vg,yjtggkz,pvdmz2l,None,exfhjex,oilfuwvu,300x50,...,Progressive Insurance,None,None,None,9,Monday: 21:00,Orlando-Daytona Beach-Melbourne FL,pkxadl3lb9,United States - DMA - Orlando-Daytona Beach-Me...,other:p9m14l1


In [8]:
df.AD_GROUP_NAME.value_counts()

Chalice LTV Display - 2023                             144692
Chalice LTV Video - 2023                                92134
Chalice LTV Native Display - 2023                       76528
Trade Desk_2023_Display_RV_Adsquare                     41832
Trade Desk_2023_Auto_Taboola Native Display Desktop     35812
                                                        ...  
Trade Desk_2023_Video_Boat_Taboola                          3
Trade Desk_2023_Display_Boat_Weather PMP                    2
Trade Desk_2023_Display_RV_Whitelist                        2
Trade Desk_2023_Display_MC_Whitelist                        1
Trade Desk_2023_Display_Boat_Lookalike                      1
Name: AD_GROUP_NAME, Length: 103, dtype: int64

In [9]:
df_video =df[df["AD_GROUP_NAME"].isin(['Chalice LTV Video - 2023', 'Chalice LTV Video - 2023 - PSA'])] 


In [10]:
df_video['y'] = np.where(df_video['AD_GROUP_NAME'] == 'Chalice LTV Video - 2023', 1 ,0)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
df_video['treat'] = np.where(df_video['CONVERSION_TYPE'] == 'other:x8cxk4v', 1 ,0)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
df_video.treat.value_counts()

1    65754
0    31089
Name: treat, dtype: int64

In [13]:
df_video.y.value_counts()

1    92134
0     4709
Name: y, dtype: int64

In [15]:
df_video = df_video[['BROWSER_NAME', 'SUPPLY_VENDOR', 'DEVICE_MAKE', 'DEVICE_MODEL', 
            'DEVICE_NAME', 'RENDERING_CONTEXT', 'REGION', 'METRO', 'ZIP_POSTAL_CODE',
            'SITE', 'OS_FAMILY_NAME', 'AD_FORMAT',
            'MATCHED_FOLD_POSITION', 'USER_HOUR_OF_WEEK','y', "FREQUENCY", "TDID", 'treat']]
df_video.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96843 entries, 9 to 999998
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   BROWSER_NAME           96843 non-null  object 
 1   SUPPLY_VENDOR          96843 non-null  object 
 2   DEVICE_MAKE            79911 non-null  object 
 3   DEVICE_MODEL           65225 non-null  object 
 4   DEVICE_NAME            96843 non-null  object 
 5   RENDERING_CONTEXT      96843 non-null  object 
 6   REGION                 96423 non-null  object 
 7   METRO                  94894 non-null  object 
 8   ZIP_POSTAL_CODE        94474 non-null  object 
 9   SITE                   96843 non-null  object 
 10  OS_FAMILY_NAME         96843 non-null  object 
 11  AD_FORMAT              96843 non-null  object 
 12  MATCHED_FOLD_POSITION  96843 non-null  object 
 13  USER_HOUR_OF_WEEK      96843 non-null  float64
 14  y                      96843 non-null  int64  
 15  F

In [16]:
df_video= df_video.dropna()
df_video


,BROWSER_NAME,SUPPLY_VENDOR,DEVICE_MAKE,DEVICE_MODEL,DEVICE_NAME,RENDERING_CONTEXT,REGION,METRO,ZIP_POSTAL_CODE,SITE,OS_FAMILY_NAME,AD_FORMAT,MATCHED_FOLD_POSITION,USER_HOUR_OF_WEEK,y,FREQUENCY,TDID,treat
9,Chrome,sharethrough,Google,Chrome,PC,Other,Colorado,773,81403,skinnyms.com,Windows,640x360,2,43.0,1,0,3673290b-d0e6-4f94-9bd5-52f0b13e29cb,0
19,Safari,yieldmo,Apple,iPhone,Mobile,MobileOptimizedWeb,Illinois,602,60602,news.yahoo.com,iOS,640x360,3,55.0,1,0,30d745a3-4590-42cd-9867-f7eb00d7e60e,1
22,Chrome,yieldmo,Samsung,SM-S908U,Mobile,MobileOptimizedWeb,Alabama,522,36874,www.the-sun.com,Android,640x360,3,61.0,1,0,3ee57bb4-36ae-4743-80db-05471af7d58d,1
23,Chrome,triplelift,Google,Chrome - Windows,PC,Other,Illinois,602,60101,mail.yahoo.com,Windows,640x360,4,61.0,1,0,069e76f0-60b0-4f7b-bcec-9f5fb47fe3fe,1
36,Chrome,yieldmo,Samsung,SM-A326U,Mobile,MobileOptimizedWeb,Delaware,504,19805,www.yahoo.com,Android,640x360,3,50.0,1,0,840b16ff-847c-402d-8e3e-6dd749ce3b87,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999937,WebView,taboola,Apple,iPhone XR,Mobile,Other,Nebraska,624,68769,www.ktiv.com,iOS,640x360,4,45.0,1,0,3c2dac35-fb90-4d13-a27f-86dce94984a5,0
999951,Chrome,yieldmo,Google,Chrome - Windows,PC,Other,Missouri,616,64148,www.yahoo.com,Windows,640x360,3,59.0,1,0,41758d4d-3133-4572-95b6-3b84ba8ac791,1
999958,Chrome,sharethrough,Samsung,SM-G986U,Mobile,Other,Illinois,602,60187,www.lakemchenryscanner.com,Android,640x360,2,61.0,1,0,5ca6fbbc-4042-40b1-b58b-89c014ac7c6a,1
999983,Edge,triplelift,Microsoft,Edge - Windows,PC,Other,Indiana,527,46255,www.msn.com,Windows,640x360,4,55.0,1,0,5127dc9c-a0af-410d-aa23-a71cfbd1da50,1


In [17]:
df_video['Counts'] = df_video.groupby(['y', "TDID"])['TDID'].transform('count')

df_video

,BROWSER_NAME,SUPPLY_VENDOR,DEVICE_MAKE,DEVICE_MODEL,DEVICE_NAME,RENDERING_CONTEXT,REGION,METRO,ZIP_POSTAL_CODE,SITE,OS_FAMILY_NAME,AD_FORMAT,MATCHED_FOLD_POSITION,USER_HOUR_OF_WEEK,y,FREQUENCY,TDID,treat,Counts
9,Chrome,sharethrough,Google,Chrome,PC,Other,Colorado,773,81403,skinnyms.com,Windows,640x360,2,43.0,1,0,3673290b-d0e6-4f94-9bd5-52f0b13e29cb,0,6
19,Safari,yieldmo,Apple,iPhone,Mobile,MobileOptimizedWeb,Illinois,602,60602,news.yahoo.com,iOS,640x360,3,55.0,1,0,30d745a3-4590-42cd-9867-f7eb00d7e60e,1,142
22,Chrome,yieldmo,Samsung,SM-S908U,Mobile,MobileOptimizedWeb,Alabama,522,36874,www.the-sun.com,Android,640x360,3,61.0,1,0,3ee57bb4-36ae-4743-80db-05471af7d58d,1,40
23,Chrome,triplelift,Google,Chrome - Windows,PC,Other,Illinois,602,60101,mail.yahoo.com,Windows,640x360,4,61.0,1,0,069e76f0-60b0-4f7b-bcec-9f5fb47fe3fe,1,33
36,Chrome,yieldmo,Samsung,SM-A326U,Mobile,MobileOptimizedWeb,Delaware,504,19805,www.yahoo.com,Android,640x360,3,50.0,1,0,840b16ff-847c-402d-8e3e-6dd749ce3b87,1,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999937,WebView,taboola,Apple,iPhone XR,Mobile,Other,Nebraska,624,68769,www.ktiv.com,iOS,640x360,4,45.0,1,0,3c2dac35-fb90-4d13-a27f-86dce94984a5,0,15
999951,Chrome,yieldmo,Google,Chrome - Windows,PC,Other,Missouri,616,64148,www.yahoo.com,Windows,640x360,3,59.0,1,0,41758d4d-3133-4572-95b6-3b84ba8ac791,1,5
999958,Chrome,sharethrough,Samsung,SM-G986U,Mobile,Other,Illinois,602,60187,www.lakemchenryscanner.com,Android,640x360,2,61.0,1,0,5ca6fbbc-4042-40b1-b58b-89c014ac7c6a,1,30
999983,Edge,triplelift,Microsoft,Edge - Windows,PC,Other,Indiana,527,46255,www.msn.com,Windows,640x360,4,55.0,1,0,5127dc9c-a0af-410d-aa23-a71cfbd1da50,1,23


In [67]:
above_35 = df_video[df_video["Counts"] < 1000000]

In [69]:
bins = [1, 2, 3, 1000000]
labels = [1,2,3]
above_35['binned'] = pd.cut(above_35['Counts'], bins=bins, labels=labels)


In [70]:
above_35.binned.value_counts()


3    53370
1     3662
2     3132
Name: binned, dtype: int64

In [71]:
above_35= above_35.dropna()
above_35.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 60164 entries, 9 to 999988
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   BROWSER_NAME           60164 non-null  object  
 1   SUPPLY_VENDOR          60164 non-null  object  
 2   DEVICE_MAKE            60164 non-null  object  
 3   DEVICE_MODEL           60164 non-null  object  
 4   DEVICE_NAME            60164 non-null  object  
 5   RENDERING_CONTEXT      60164 non-null  object  
 6   REGION                 60164 non-null  object  
 7   METRO                  60164 non-null  object  
 8   ZIP_POSTAL_CODE        60164 non-null  object  
 9   SITE                   60164 non-null  object  
 10  OS_FAMILY_NAME         60164 non-null  object  
 11  AD_FORMAT              60164 non-null  object  
 12  MATCHED_FOLD_POSITION  60164 non-null  object  
 13  USER_HOUR_OF_WEEK      60164 non-null  float64 
 14  y                      60164 non-null

In [73]:
# X = df.drop(['IN_VIEW_5S', "AD_SIZE", "IN_VIEW_FIELD",
#             'IN_VIEW_15S', 'FULLY_IN_VIEW_1S', 'FULLY_IN_VIEW_5S', 'VIDEO_AD_1Q_IN_VIEW', 'VIDEO_AD_2Q_IN_VIEW',
#             'VIDEO_AD_3Q_IN_VIEW', 'VIDEO_AD_4Q_IN_VIEW','y'], axis=1)
X = above_35[['BROWSER_NAME', 'SUPPLY_VENDOR', 'DEVICE_MAKE', 'DEVICE_MODEL', 
            'DEVICE_NAME', 'RENDERING_CONTEXT', 'REGION', 'METRO', 'ZIP_POSTAL_CODE',
            'SITE', 'OS_FAMILY_NAME', 'AD_FORMAT',
            'MATCHED_FOLD_POSITION', 'USER_HOUR_OF_WEEK']]
Y = above_35['y']
treat = above_35['treat']

In [74]:
X_train, X_val, trmnt_train, trmnt_val, y_train, y_val = train_test_split(
    X,
    treat,
    Y,
    test_size=0.2,
    random_state=42
)



In [75]:

categorical_features_indices = np.where(X.dtypes != np.float64)[0]
print(categorical_features_indices)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [76]:
from scipy import stats


In [77]:
base = above_35[above_35.treat == 0]
variant =above_35[above_35.treat == 1]
    
delta = variant.y.mean() - base.y.mean()

delta_err = 1.96 * np.sqrt(variant.y.var() / variant.shape[0] + base.y.var() / base.shape[0])
  
print("Control: %s, N = %s, Conversion Rate: %s" % (above_35[above_35.treat == 0]['y'].sum(), above_35[above_35.treat == 0].shape[0], 
                                         above_35[above_35.treat == 0]['y'].sum() / float(above_35[above_35.treat == 0].shape[0])))

print("\nTest: %s, N = %s, Conversion Rate: %s" % (above_35[above_35.treat == 1]['y'].sum(), above_35[above_35.treat == 1].shape[0], 
                                      above_35[above_35.treat == 1]['y'].sum() / float(above_35[above_35.treat == 1].shape[0])))

print("\nEstimated Effect %s, Standard Error: %s" % (delta, delta_err))

lift = delta / base.y.mean()
print("\nLift: %s" % lift)

# Run a t-test between the exposed and control group
t = stats.ttest_ind(above_35[above_35.treat == 1]['y'], above_35[above_35.treat == 0]['y'])
print("\nTest Statistic: %s, P-Value: %s" % (t[0], t[1]))

Control: 16424, N = 17606, Conversion Rate: 0.9328637964330342

Test: 40202, N = 42558, Conversion Rate: 0.9446402556511114

Estimated Effect 0.01177645921807724, Standard Error: 0.004288001187113877

Lift: 0.012623985691273008

Test Statistic: 5.587565421565756, P-Value: 2.3126853752041215e-08


In [78]:
exp_results_df = \
above_35.groupby('treat').agg({'y':['mean', 'sum', 'count']})
exp_results_df


y              
           mean    sum  count
treat                        
0      0.932864  16424  17606
1      0.944640  40202  42558